In [1]:
# Step 1: Imports and Setup
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from bs4 import BeautifulSoup
from sklearn.metrics import classification_report, confusion_matrix

nltk.download('punkt_tab', force=True)   # will re-download even if it thinks it's there
nltk.download('punkt', force=True)

# Download NLTK resources if needed
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install wandb -q
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jarrennsb (jarrennsb-abc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
# Step 2: Load Data
# Assuming data is in 'spam_assassin.csv' - adjust path if needed
df = pd.read_csv("spam_assassin.csv", encoding="utf-8")
y = df["target"].values
print(df.info())
print(df.head())

# Class distribution
print(df["target"].value_counts())

# Length features (optional, not used in model)
df["char_len"] = df["text"].astype(str).str.len()
df["word_len"] = df["text"].astype(str).apply(lambda x: len(x.split()))
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5796 entries, 0 to 5795
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5796 non-null   object
 1   target  5796 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 90.7+ KB
None
                                                text  target
0  From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...       0
1  From gort44@excite.com Mon Jun 24 17:54:21 200...       1
2  From fork-admin@xent.com Mon Jul 29 11:39:57 2...       1
3  From dcm123@btamail.net.cn Mon Jun 24 17:49:23...       1
4  From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...       0
target
0    3900
1    1896
Name: count, dtype: int64
                                                text  target  char_len  \
0  From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...       0      4098   
1  From gort44@excite.com Mon Jun 24 17:54:21 200...       1      2195   
2  From fork-admin@xent.com Mon Jul 29 11:39:57 2... 

In [4]:
# Step 3: Improved Text Cleaning Function
def clean_text(text):
    # 1. Decode HTML entities & remove tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # 2. DETECT PHISHING PATTERNS FIRST (before URL extraction!)
    def detect_phishing_patterns(text):
        """Detect common phishing patterns"""
        text_lower = text.lower()

        # Phishing keywords
        phishing_words = ['verify', 'suspended', 'confirm', 'urgent', 'action required',
                          'update payment', 'security alert', 'unusual activity']

        # Check for phishing + URL combination
        has_url = bool(re.search(r'http[s]?://|www\.', text))
        has_phishing_word = any(word in text_lower for word in phishing_words)

        # If phishing word + URL, add token
        if has_url and has_phishing_word:
            text = 'PHISHING ' + text

        return text

    # ✅ RUN PHISHING DETECTION BEFORE URL EXTRACTION
    text = detect_phishing_patterns(text)

    # 3. Extract URL domain features (AFTER phishing detection)
    def extract_url_features(text):
        """Extract domain components instead of replacing with generic <URL>"""
        urls = re.findall(r'(http[s]?://[^\s]+|www\.[^\s]+)', text, flags=re.IGNORECASE)

        for url in urls:
            domain_match = re.search(r'(?:http[s]?://)?([^/\s]+)', url)

            if domain_match:
                domain = domain_match.group(1)
                domain = re.sub(r'^www\.', '', domain, flags=re.IGNORECASE)
                domain_parts = domain.replace('.', ' ').replace('-', ' ')
                replacement = f' url {domain_parts} '
                text = text.replace(url, replacement)

        return text

    text = extract_url_features(text)

    # 4. Email and phone replacement
    text = re.sub(r'\S+@\S+', 'EMAIL', text)
    text = re.sub(r'\b(?:\+?\d{1,3}[-.\s]?)?(?:\(\d{3}\)|\d{3})[-.\s]?\d{3}[-.\s]?\d{4}\b', ' <PHONE> ', text)

    # 5. Detect large money amounts
    def detect_large_amounts(text):
        amounts = re.findall(r'\$[\d,]+', text)
        for amount in amounts:
            num_str = amount.replace('$', '').replace(',', '')
            if num_str.isdigit():
                num = int(num_str)
                if num >= 10000:
                    text = text.replace(amount, ' XXLARGEMONEY ')
                elif num >= 100:
                    text = text.replace(amount, ' XXMONEY ')
        return text

    text = detect_large_amounts(text)

    # 6. Detect phishing patterns
    def detect_phishing_patterns(text):
        text_lower = text.lower()
        phishing_words = ['verify', 'suspended', 'confirm', 'urgent', 'action required',
                          'update payment', 'security alert', 'unusual activity']
        has_url = bool(re.search(r'http[s]?://|www\.', text))
        has_phishing_word = any(word in text_lower for word in phishing_words)

        if has_url and has_phishing_word:
            text = 'PHISHING ' + text  # ✅ No brackets!

        return text

    text = detect_phishing_patterns(text)



    # 7. Lowercase
    text = text.lower()

    # 7a. Replace placeholders with lowercase tokens
    text = text.replace('xxlargemoney', 'largemoney')
    text = text.replace('xxmoney', 'money')

    # 7b. Phishing detection AFTER lowercasing
    phishing_words = ['verify', 'suspended', 'confirm', 'urgent', 'action required',
                      'update payment', 'security alert', 'unusual activity']
    has_url = bool(re.search(r'url', text))  # Already processed
    has_phishing_word = any(word in text for word in phishing_words)

    if has_url and has_phishing_word:
        text = 'phishing ' + text  # lowercase token with space

    # 8. ✅ NEW: Prize scam detection
    prize_words = ['won', 'win', 'winner', 'prize', 'congratulations', 'claim']
    has_large_money = 'largemoney' in text
    has_prize_word = any(word in text for word in prize_words)

    if has_large_money and has_prize_word:
        text = 'prizescam ' + text  # ✅ Add prize scam token!




    # 9. Normalize repeated punctuation
    text = re.sub(r'([!?.]){3,}', r'\1\1\1', text)

    # 10. Collapse multiple spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply cleaning
df["clean_text"] = df["text"].astype(str).apply(clean_text)
print("\nSample cleaned text:\n", df["clean_text"][0][:500], "...")


Sample cleaned text:
 from email mon jul 29 11:28:02 2002 return-path: delivered-to: email received: from localhost (localhost [127.0.0.1]) by phobos.labs.netnoteinc.com (postfix) with esmtp id a13d94414f for ; mon, 29 jul 2002 06:25:11 -0400 (edt) received: from phobos [127.0.0.1] by localhost with imap (fetchmail-5.9.0) for email (single-drop); mon, 29 jul 2002 11:25:11 +0100 (ist) received: from lugh.tuatha.org email [194.125.145.45]) by dogma.slashnull.org (8.11.6/8.11.6) with esmtp id g6rhn7i17130 for ; sat, 27  ...


In [5]:
# Step 4: Tokenization (Light Stopword Removal - Optional, but recommended light version)
stop_words = set(stopwords.words('english')) - {'not', 'no', 'never', 'none'}  # Keep negation words for sentiment/spam

def tokenize(text):
    tokens = word_tokenize(text)
    # Light stopword removal: Only remove very common ones if desired
    # Comment out the next line to skip stopword removal entirely
    tokens = [word for word in tokens if word not in list(stop_words)[:50]]  # e.g., top 50 stopwords only
    return tokens

# Build vocabulary
all_tokens = []
for text in df["clean_text"]:
    all_tokens.extend(tokenize(text))

word_counts = Counter(all_tokens)
vocab_size = 15000
vocab = sorted(word_counts, key=word_counts.get, reverse=True)[:vocab_size-7]  # Reserve for specials

specials = ["<PAD>", "<UNK>", "<START>","phishing", "largemoney", "money", "prizescam"]
stoi = {word: idx + len(specials) for idx, word in enumerate(vocab)}
stoi.update({special: idx for idx, special in enumerate(specials)})

itos = specials + vocab

print(f"Vocabulary size: {len(stoi)}")
print("Sample vocab:", list(stoi.keys())[:10])

# Verify special tokens are in vocabulary
print("\n" + "="*70)
print("VERIFYING SPECIAL TOKENS")
print("="*70)
for token in specials:
    if token in stoi:
        print(f"✓ {token:20s} → index {stoi[token]}")
    else:
        print(f"✗ {token:20s} → MISSING!")
print("="*70)

Vocabulary size: 14996
Sample vocab: [':', ',', ')', '(', 'email', '--', '.', 'from', '2002', "''"]

VERIFYING SPECIAL TOKENS
✓ <PAD>                → index 0
✓ <UNK>                → index 1
✓ <START>              → index 2
✓ phishing             → index 3
✓ largemoney           → index 4
✓ money                → index 5
✓ prizescam            → index 6


Check for spam keywords

In [6]:
spam_keywords = ['win', 'won', 'prize', 'claim', 'free', 'urgent',
                 'limited', 'act', 'click', 'verify', 'congratulations']

for word in spam_keywords:
    if word in stoi:
        print(f"✓ '{word}' in vocab (index {stoi[word]})")
    else:
        print(f"✗ '{word}' NOT in vocab")

✓ 'win' in vocab (index 1511)
✗ 'won' NOT in vocab
✓ 'prize' in vocab (index 3413)
✓ 'claim' in vocab (index 979)
✓ 'free' in vocab (index 113)
✓ 'urgent' in vocab (index 1736)
✓ 'limited' in vocab (index 999)
✓ 'act' in vocab (index 769)
✓ 'click' in vocab (index 167)
✓ 'verify' in vocab (index 2455)
✓ 'congratulations' in vocab (index 3604)


In [7]:
# Step 5: Encoding and Padding
max_len = 200

def encode(text, stoi, max_len=max_len):
    tokens = tokenize(text)
    ids = [stoi.get(tok, stoi["<UNK>"]) for tok in tokens]
    ids = ids[:max_len]
    if len(ids) < max_len:
        ids = ids + [stoi["<PAD>"]] * (max_len - len(ids))
    return np.array(ids, dtype=np.int64)

In [8]:
# Step 6: Data Split
texts = df["clean_text"].values
X_train, X_temp, y_train, y_temp = train_test_split(texts, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

X_train_ids = np.stack([encode(t, stoi) for t in X_train])
X_val_ids   = np.stack([encode(t, stoi) for t in X_val])
X_test_ids  = np.stack([encode(t, stoi) for t in X_test])

y_train = y_train.astype(np.float32)
y_val   = y_val.astype(np.float32)
y_test  = y_test.astype(np.float32)

Train: 4057, Val: 869, Test: 870


In [9]:
# Step 7: Datasets and Loaders
class SpamDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 32
train_ds = SpamDataset(X_train_ids, y_train)
val_ds   = SpamDataset(X_val_ids, y_val)
test_ds  = SpamDataset(X_test_ids, y_test)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [10]:
# Step 8: Model Definition
class ImprovedSPAMGRU(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, hidden_size=256, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # Bidirectional GRU
        self.gru = nn.GRU(
            embed_dim,
            hidden_size,
            num_layers=2,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )

        # Attention mechanism
        self.attention = nn.Linear(hidden_size * 2, 1)

        # Classification layers
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Embedding
        embedded = self.embedding(x)

        # GRU
        gru_out, _ = self.gru(embedded)

        # Attention
        attention_weights = torch.softmax(self.attention(gru_out), dim=1)
        context = torch.sum(attention_weights * gru_out, dim=1)

        # Classification
        x = self.fc1(context)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return self.sigmoid(x).squeeze()

In [11]:
# Step 9: Training Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

EMBED_DIM = 256
HIDDEN_SIZE = 256
DROPOUT = 0.3
NUM_EPOCHS = 10
LEARNING_RATE = 0.001

model = ImprovedSPAMGRU(
    vocab_size=len(stoi),
    embed_dim=EMBED_DIM,
    hidden_size=HIDDEN_SIZE,
    dropout=DROPOUT
).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training function
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for texts, labels in train_loader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Evaluation function
def evaluate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for texts, labels in val_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            loss = criterion(outputs, labels.float())
            total_loss += loss.item()
            predictions = (outputs >= 0.5).long()
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    avg_loss = total_loss / len(val_loader)
    return avg_loss, accuracy

Using device: cuda


In [12]:
# Initialize W&B project
wandb.init(
    project="spam-detection",
    name="ImprovedSPAMGRU-BiGRU",
    config={
        "epochs": NUM_EPOCHS,
        "learning_rate": LEARNING_RATE,
        "vocab_size": len(stoi),
        "embed_dim": EMBED_DIM,
        "hidden_size": HIDDEN_SIZE,
        "dropout": DROPOUT,
    }
)

print("✅ W&B initialized! View your run at:", wandb.run.get_url())

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


✅ W&B initialized! View your run at: https://wandb.ai/jarrennsb-abc/spam-detection/runs/krmijcsp


In [13]:
# Step 10: Training Loop
print("=" * 70)
print("TRAINING IMPROVEDSPAMGRU MODEL")
print("=" * 70)

best_val_acc = 0
for epoch in range(NUM_EPOCHS):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    # ADDED LINE - Log to W&B
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    })


    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}")
    print(f"  Val Accuracy: {val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'spam_model.pth')
        print("  ✓ New best model saved!")
    print()

print("=" * 70)
print("TRAINING COMPLETE!")
print(f"Best Validation Accuracy: {best_val_acc:.4f}")
print("=" * 70)

TRAINING IMPROVEDSPAMGRU MODEL
Epoch 1/10
  Train Loss: 0.2297
  Val Loss: 0.1108
  Val Accuracy: 0.9586
  ✓ New best model saved!

Epoch 2/10
  Train Loss: 0.0822
  Val Loss: 0.1464
  Val Accuracy: 0.9379

Epoch 3/10
  Train Loss: 0.0396
  Val Loss: 0.0510
  Val Accuracy: 0.9850
  ✓ New best model saved!

Epoch 4/10
  Train Loss: 0.0241
  Val Loss: 0.0417
  Val Accuracy: 0.9839

Epoch 5/10
  Train Loss: 0.0203
  Val Loss: 0.0472
  Val Accuracy: 0.9781

Epoch 6/10
  Train Loss: 0.0107
  Val Loss: 0.0357
  Val Accuracy: 0.9827

Epoch 7/10
  Train Loss: 0.0126
  Val Loss: 0.0335
  Val Accuracy: 0.9873
  ✓ New best model saved!

Epoch 8/10
  Train Loss: 0.0023
  Val Loss: 0.0415
  Val Accuracy: 0.9827

Epoch 9/10
  Train Loss: 0.0052
  Val Loss: 0.0571
  Val Accuracy: 0.9839

Epoch 10/10
  Train Loss: 0.0149
  Val Loss: 0.0641
  Val Accuracy: 0.9804

TRAINING COMPLETE!
Best Validation Accuracy: 0.9873


In [14]:
# Step 11: Test Evaluation and Classification Report
model.load_state_dict(torch.load('spam_model.pth'))
test_loss, test_acc = evaluate(model, test_loader, criterion, device)

print("\nFinal Test Results:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Accuracy: {test_acc:.4f}")

# Log training loss and accuracy
wandb.summary["test_loss"] = test_loss
wandb.summary["test_accuracy"] = test_acc

# Detailed classification report
model.eval()
all_predictions = []
all_labels = []
with torch.no_grad():
    for texts, labels in test_loader:
        texts = texts.to(device)
        outputs = model(texts)
        predictions = (outputs >= 0.5).long().cpu().numpy()
        all_predictions.extend(predictions)
        all_labels.extend(labels.numpy())

print("\n" + "=" * 70)
print("CLASSIFICATION REPORT")
print("=" * 70)
print(classification_report(all_labels, all_predictions,
                            target_names=['Ham (Non-Spam)', 'Spam'],
                            digits=4))

print("\n" + "=" * 70)
print("CONFUSION MATRIX")
print("=" * 70)
cm = confusion_matrix(all_labels, all_predictions)
print(f"                  Predicted")
print(f"                  Ham    Spam")
print(f"Actual Ham     {cm[0][0]:6d} {cm[0][1]:6d}")
print(f"       Spam    {cm[1][0]:6d} {cm[1][1]:6d}")
print("=" * 70)

# Model summary
print("\nModel Architecture:")
print(model)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal Parameters: {total_params:,}")


Final Test Results:
  Test Loss: 0.0367
  Test Accuracy: 0.9874

CLASSIFICATION REPORT
                precision    recall  f1-score   support

Ham (Non-Spam)     0.9983    0.9829    0.9905       585
          Spam     0.9660    0.9965    0.9810       285

      accuracy                         0.9874       870
     macro avg     0.9821    0.9897    0.9858       870
  weighted avg     0.9877    0.9874    0.9874       870


CONFUSION MATRIX
                  Predicted
                  Ham    Spam
Actual Ham        575     10
       Spam         1    284

Model Architecture:
ImprovedSPAMGRU(
  (embedding): Embedding(14996, 256, padding_idx=0)
  (gru): GRU(256, 256, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (attention): Linear(in_features=512, out_features=1, bias=True)
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

Tota

In [15]:
# Log final test results to W&B
wandb.log({
    "test_loss": test_loss,
    "test_accuracy": test_acc
})

print(f"✅ Test results logged to W&B!")
print(f"   Test Loss: {test_loss:.4f}")
print(f"   Test Accuracy: {test_acc:.4f}")

✅ Test results logged to W&B!
   Test Loss: 0.0367
   Test Accuracy: 0.9874


In [16]:
# Finish W&B run
wandb.finish()
print("✅ W&B run finished! Check your dashboard at: https://wandb.ai")

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
test_loss,▁
train_loss,█▃▂▂▂▁▁▁▁▁
val_accuracy,▄▁██▇▇█▇█▇
val_loss,▆█▂▂▂▁▁▁▂▃
epoch,10
test_accuracy,0.98736
test_loss,0.0367
train_loss,0.01487
val_accuracy,0.98044


✅ W&B run finished! Check your dashboard at: https://wandb.ai


In [17]:
# Step 12: Save Model and Config for Deployment

config = {
    'vocab_size': len(stoi),
    'embed_dim': EMBED_DIM,
    'hidden_size': HIDDEN_SIZE,
    'dropout': DROPOUT,
    'max_len': max_len,
    'stoi': stoi
}
torch.save(config, 'spam_config.pth')
print("✓ Model and configuration saved!")
print("  - spam_model.pth")
print("  - spam_config.pth")

✓ Model and configuration saved!
  - spam_model.pth
  - spam_config.pth


In [18]:
# ============================================================================
# UPDATED DIAGNOSTIC TEST
# ============================================================================

import os
import torch

print("="*80)
print("COMPREHENSIVE MODEL DIAGNOSTIC TEST")
print("="*80)

# Test 1: Check if special tokens are in vocabulary
print("\n📚 TEST 1: Special Tokens in Vocabulary")
print("-"*80)
special_tokens_needed = ['phishing', 'largemoney', 'money','prizescam']  # ✅ Updated names
missing_tokens = []

for token in special_tokens_needed:
    if token in stoi:
        print(f"✓ {token:25s} → index {stoi[token]}")
    else:
        print(f"✗ {token:25s} → MISSING IN VOCABULARY!")
        missing_tokens.append(token)

if missing_tokens:
    print(f"\n❌ CRITICAL: {len(missing_tokens)} special tokens missing!")
    print("   These tokens will be mapped to <UNK> and model can't learn from them!")
    print("   FIX: Update Cell 3 to include these in specials list, then RETRAIN")
else:
    print("\n✅ All special tokens present in vocabulary")

# Test 2: Check preprocessing on spam examples
print("\n\n🧹 TEST 2: Preprocessing Output")
print("-"*80)

test_examples = [
    ("Prize scam", "You've just won $2,500,000! Claim now: http://luckyclaim.net"),
    ("Phishing", "URGENT: Account suspended! Verify now: http://secure-bank-login.xyz"),
    ("Free offer", "CONGRATULATIONS!!! FREE iPhone 15! Click: http://winner-prize.com LIMITED TIME!!!")
]

for name, text in test_examples:
    cleaned = clean_text(text)
    tokens = tokenize(cleaned)

    print(f"\n{name}:")
    print(f"  Cleaned: {cleaned[:100]}...")
    print(f"  First tokens: {tokens[:10]}")

    # ✅ Check for NEW token names (no brackets!)
    has_phishing = 'phishing' in cleaned.lower()
    has_large_money = 'largemoney' in cleaned.lower()
    has_money = 'money' in cleaned.lower() and 'largemoney' not in cleaned.lower()

    signals = []
    if has_phishing:
        signals.append("🎣 PHISHING")
    if has_large_money:
        signals.append("💰 LARGEMONEY")
    if has_money:
        signals.append("💵 MONEY")

    if signals:
        print(f"  Signals: {', '.join(signals)}")
    else:
        print(f"  ❌ NO SPECIAL TOKENS DETECTED!")

# Test 3: Encode and check indices
print("\n\n🔢 TEST 3: Token Encoding")
print("-"*80)

test_text = "You've just won $2,500,000! Claim now: http://luckyclaim.net"
cleaned = clean_text(test_text)
tokens = tokenize(cleaned)
encoded = encode(cleaned, stoi)

print(f"Example: {test_text[:60]}...")
print(f"Cleaned: {cleaned[:80]}...")
print(f"Tokens: {tokens[:10]}")
print(f"Encoded (first 15): {encoded[:15]}")

# ✅ Check for NEW special token indices
special_indices = {stoi.get(t, -1) for t in ['phishing', 'largemoney', 'money']}
found_specials = [idx for idx in encoded[:20] if idx in special_indices and idx != -1]

if found_specials:
    print(f"✓ Special token indices found: {found_specials}")
else:
    print(f"❌ NO special token indices in encoded sequence!")

# Test 4: Test model prediction
print("\n\n🤖 TEST 4: Model Predictions")
print("-"*80)

test_cases = [
    ("Prize scam", "You've just won $2,500,000! Claim now: http://luckyclaim.net", "SPAM"),
    ("Phishing", "URGENT: Account suspended! Verify now: http://secure-bank-login.xyz", "SPAM"),
    ("Free iPhone", "CONGRATULATIONS!!! FREE iPhone 15! Click: http://winner-prize.com LIMITED TIME!!!", "SPAM"),
    ("Legitimate", "Meeting tomorrow at 2 PM. Please review the documents.", "HAM")
]

correct = 0
total = len(test_cases)

for name, text, expected in test_cases:
    cleaned = clean_text(text)
    encoded = encode(cleaned, stoi)
    x = torch.tensor([encoded]).to(device)

    with torch.no_grad():
        prob = model(x).item()

    predicted = "SPAM" if prob >= 0.5 else "HAM"
    is_correct = (predicted == expected)

    symbol = "✓" if is_correct else "✗"
    if is_correct:
        correct += 1

    print(f"{symbol} {name:15s} Expected: {expected:4s}, Got: {predicted:4s} ({prob*100:5.1f}% spam)")

print(f"\nAccuracy on test cases: {correct}/{total} ({correct/total*100:.0f}%)")

if correct < 3:
    print("\n❌ CRITICAL FAILURE: Model performing poorly!")
    print("   Most likely causes:")
    print("   1. Special tokens not in vocabulary (check TEST 1)")
    print("   2. Using old model (not retrained after fixing preprocessing)")
    print("   3. Need more training data with these spam types")

# Test 5: Check saved config
print("\n\n💾 TEST 5: Saved Configuration")
print("-"*80)

if os.path.exists('spam_config.pth'):
    saved_config = torch.load('spam_config.pth', map_location='cpu')

    print(f"✓ Config file exists")
    print(f"  Vocab size: {len(saved_config['stoi'])}")

    # ✅ Check for NEW token names
    saved_specials = [t for t in ['phishing', 'largemoney', 'money', 'prizescam']
                      if t in saved_config['stoi']]

    if len(saved_specials) == 4:
        print(f"✓ All 4 special tokens in saved config")
        for token in saved_specials:
            print(f"  {token} → index {saved_config['stoi'][token]}")
    else:
        print(f"❌ Only {len(saved_specials)}/3 special tokens in saved config!")
        print(f"   Found: {saved_specials}")
else:
    print("❌ Config file not found!")

print("\n" + "="*80)
print("DIAGNOSTIC TEST COMPLETE")
print("="*80)

if missing_tokens:
    print("\n🚨 ACTION REQUIRED:")
    print("1. Update Cell 3 to include special tokens in vocabulary")
    print("2. Restart kernel")
    print("3. Run ALL cells again to retrain")
    print("4. Download NEW model files")
    print("5. Upload to Hugging Face")
elif correct < 3:
    print("\n🚨 ACTION REQUIRED:")
    print("Model not learning spam patterns properly.")
    print("Check if training data has enough spam examples of these types.")
else:
    print("\n✅ Model looks good! Ready for deployment.")
    print("   Download spam_model.pth and spam_config.pth")
    print("   Upload to Hugging Face")

print("="*80)

COMPREHENSIVE MODEL DIAGNOSTIC TEST

📚 TEST 1: Special Tokens in Vocabulary
--------------------------------------------------------------------------------
✓ phishing                  → index 3
✓ largemoney                → index 4
✓ money                     → index 5
✓ prizescam                 → index 6

✅ All special tokens present in vocabulary


🧹 TEST 2: Preprocessing Output
--------------------------------------------------------------------------------

Prize scam:
  Cleaned: prizescam you've just won largemoney ! claim now: url luckyclaim net...
  First tokens: ['prizescam', 'you', "'ve", 'just', 'largemoney', '!', 'claim', ':', 'url', 'luckyclaim']
  Signals: 💰 LARGEMONEY

Phishing:
  Cleaned: phishing phishing urgent: account suspended! verify now: url secure bank login xyz...
  First tokens: ['phishing', 'phishing', 'urgent', ':', 'account', 'suspended', '!', 'verify', ':', 'url']
  Signals: 🎣 PHISHING

Free offer:
  Cleaned: congratulations!!! free iphone 15! click: url 

/tmp/ipython-input-3046786870.py:108: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  x = torch.tensor([encoded]).to(device)
